In [1]:
!pip install pytubefix
!pip install google-generativeai==0.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.4/725.4 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.6
    Uninstalling google-ai-generativelanguage-0.6.6:
      Successfully uninstalled google-ai-generativelanguage-0.6.6
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.7.2
    Uninstalling google-generativeai-0.7.2:
      Successfully uninstalled google-generativeai-0.7.2


In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab Notebooks/5_gemini/data")

Mounted at /content/drive


In [2]:
# @title 유튜브 동영상 다운로드

from pytubefix import YouTube
url = "https://www.youtube.com/watch?v=i-E7NiyRDa0"

yt = YouTube(url) # YouTube 객체 생성
stream = yt.streams.get_highest_resolution() # 가장 높은 해상도의 스트림 선택

# 동영상 다운로드
file_path = stream.download(output_path="./videos")

In [3]:
# @title 동영상 업로드하기
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key= userdata.get('GOOGLE_API_KEY'))
uploaded_file = genai.upload_file(path=file_path)
print("uploaded_file.uri:", uploaded_file.uri)

uploaded_file.uri: https://generativelanguage.googleapis.com/v1beta/files/ivwhb8bagg9b


In [4]:
# @title 동영상 인식하기
import IPython.display  # 동영상 플레이어 출력을 위해 추가

model = genai.GenerativeModel(model_name="gemini-1.5-flash")
prompt = """
유튜브를 보고 아래에 답하세요.
- 영상에 등장하는 춤을 추는 인물은 몇 명인가요?
- 각각의 인물에 대한 특징을 짧게 기술하세요.
"""
contents = [prompt, uploaded_file]
responses = model.generate_content(contents, stream=True, request_options={"timeout": 60*2})

IPython.display.display(IPython.display.Video(file_path, width=800 ,embed=True))
for response in responses:
    print(response.text.strip(), end="")


In [5]:
# @title 동영상 삭제하기
if os.path.exists(file_path):
  os.remove(file_path)
  uploaded_file.delete()

In [ ]:
# @title 유튜브 영상 인식 파이프라인 만들기
def download_youtube(url):
    yt = YouTube(url) # YouTube 객체 생성
    stream = yt.streams.get_highest_resolution() # 가장 높은 해상도의 스트림 선택
    # 현재 디렉터리에 동영상 다운로드
    file_path = stream.download(output_path="./videos")
    print("Download complete!")
    return file_path

def delete_file(file_path, uploaded_file):
    if os.path.exists(file_path):
      os.remove(file_path)
      uploaded_file.delete()
      print("Delete complete!")

def recog_video(prompt, url, model):
  file_path = download_youtube(url)
  uploaded_file = genai.upload_file(path=file_path)
  contents = [prompt, uploaded_file]
  responses = model.generate_content(contents, stream=True, request_options={"timeout": 60*2})

  IPython.display.display(IPython.display.Video(file_path, width=800 ,embed=True))
  for response in responses:
      print(response.text.strip(), end="")

  delete_file(file_path, uploaded_file)

model = genai.GenerativeModel(model_name="gemini-1.5-flash")

prompt = """
- 주인공과 영상의 배경을 소설처럼 디테일하게 묘사하세요.
- 주인공이 어느 도시에 있는지 말하고, 왜 그렇게 생각하는지 설명하세요.
- 만일 이 영상이 생성형 AI가 만들었다면, 어떤 부분이 가장 놀랍나요?
"""
url = "https://www.youtube.com/watch?v=7qQTyBW4uhI"
recog_video(prompt, url, model)


In [ ]:
from datetime import datetime
import pytz

In [ ]:
def convert_to_kst(utc_time):
    fmt_utc_time = datetime.fromisoformat(str(utc_time))
    kst = pytz.timezone("Asia/Seoul")
    kst_time = fmt_utc_time.astimezone(kst)
    return kst_time.strftime("%Y-%m-%d %H:%M:%S.%f")

model = genai.GenerativeModel('gemini-1.5-flash-latest')
path = './audios/5-things-to-know-from-GoogleIO-2024.mp3'

# File API로 오디오 파일 업로드
audio_file = genai.upload_file(path=path)

# 프롬프트에 오디오 파일 포함하여 응답 생성
response = model.generate_content(["한국어로 요약해줘.", audio_file])
print(response.text)
print(f"생성시각: {convert_to_kst(audio_file.create_time)}")
print(f"만료시각: {convert_to_kst(audio_file.expiration_time)}")
audio_file.delete()


구글 I/O에서 놓치셨을 수도 있는 5가지 소식을 알려드립니다.

* **Gemini 1.5 Flash** :  빠르고 저렴한 작업을 위한 경량 모델이 출시되었습니다.
* **Vyo** : 1분 이상의 고품질 영상을 생성하는 텍스트 기반 비디오 모델을 소개합니다.
* **Gemini 1.5 Pro** :  워크스페이스와 Gemini Advanced에서 사용할 수 있습니다. 더 넓은 컨텍스트 창을 통해 복잡한 문제 해결에 도움을 줍니다.
* **AI 오버뷰** :  미국에서 모든 사람이 사용할 수 있는 검색 기능입니다. AI가 복잡한 질문에 답변하고 작업을 완료하도록 도와줍니다.
* **Ask Photos** :  Gemini 모델을 활용하여 Google Photos에서 검색 기능을 향상시킨 새로운 기능입니다. 갤러리에 숨겨진 정보를 찾는 데 도움이 됩니다.

자세한 내용은 blog.google에서 확인하세요. 

생성시각: 2024-09-17 02:31:33.620236
만료시각: 2024-09-19 02:31:33.600301
